In [1]:
import argparse
import logging
import random
import time
import os
from typing import List, Optional, Tuple, Union
import copy

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from transformers import AdamW, get_linear_schedule_with_warmup

from utils import get_classifier, generate_next, concat_past, expand_past


In [2]:
seed = 0
device = "cuda"

pretrained_model = "gpt2-medium"
discrim = "sentiment"
class_label = 3
num_of_triggers = 1

trigger_format = "key_value"  # token or key_value
TRIGGER_POSITION_ID = 0  # for position reset

num_iterations = 2
learning_rate = 5e-3
# learning_rate = 0
gradient_accumulation_steps = 1

sample = True
gumbel_softmax = True
detach = True
# ----------------------
reset_pos_emb = True
not_mask_trigger = False
gumbel_temperature = 1.0

batch_size = 4
multiple_input = True

# more or less fixed for generation
top_k = 10
temperature = 1.0
repetition_penalty = 1.0
adam_epsilon = 1e-8
max_grad_norm = 1.0
length = 40

if detach and not gumbel_softmax:
    assert False, "require gumbel softmax when using detach"

In [3]:
torch.manual_seed(seed)
np.random.seed(seed)

# load pretrained model
model = GPT2LMHeadModel.from_pretrained(pretrained_model, output_hidden_states=True)
model.to(device)
model.eval()  # do not need batchnorm or dropout layers for training/eval
# ori_model.eval()

# load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model)

# Freeze GPT-2 weights
for param in model.parameters():
    param.requires_grad = False

classifier, class_id = get_classifier(discrim, class_label, device)

num_layers = model.config.n_layer

ce_loss = nn.CrossEntropyLoss()

In [4]:
# cond_text_list = ["This is a terrible restaurant.", "I like drinking water.", "It is raining today.", "I'm doing my homework."]
cond_text_list = ["My favorite music genre is death metal.", "I listen to rap music.", "I workout four hours a day.", "I'm a christian."]
# cond_text_list = ["I don't like this restaurant."]

In [5]:
if multiple_input:
    # TODO: implement dataloader to get a batch of cond_text
    batch_max_length = 0
    batch_min_length = 10000
    batch_input_ids = list()
    all_inputs, all_attention_masks, all_lengths, all_padding_length = list(), list(), list(), list()
    padding_token = tokenizer.encode(tokenizer.bos_token)[0]  # WARNING: BOS is for GPT2 only. Should use padding token
    for cond_text in cond_text_list[:batch_size]:
        inputs_ids = tokenizer.encode(tokenizer.bos_token + cond_text)
        batch_max_length = len(inputs_ids) if len(inputs_ids) > batch_max_length else batch_max_length
        batch_min_length = len(inputs_ids) if len(inputs_ids) < batch_min_length else batch_min_length
        batch_input_ids.append(inputs_ids)
    for inputs_ids in batch_input_ids:
        all_lengths.append(len(inputs_ids))
        padding_len = batch_max_length - len(inputs_ids)
#         attention_mask = [0] * padding_len + [1] * len(inputs_ids)
#         inputs_ids = [padding_token] * padding_len + inputs_ids
        attention_mask = [1] * len(inputs_ids) + [0] * padding_len
        inputs_ids = inputs_ids + [padding_token] * padding_len 
        
        all_padding_length.append(padding_len)
        all_inputs.append(inputs_ids)
        all_attention_masks.append(attention_mask)
        
    all_input_ids = torch.tensor(all_inputs, dtype=torch.long, device=device)
    all_attention_masks = torch.tensor(all_attention_masks, dtype=torch.long, device=device)
        
else:    
    cond_text = "Today is Monday."
    tokenized_cond_text = tokenizer.encode(tokenizer.bos_token + cond_text)

    context_t = torch.tensor(tokenized_cond_text, device=device, dtype=torch.long)
    while len(context_t.shape) < 2:
        context_t = context_t.unsqueeze(0)
    input_length = context_t.shape[-1]
    all_input_ids = context_t.repeat(batch_size, 1).to(device)
    all_attention_masks = torch.ones(batch_size, input_length).to(torch.long).to(device)
    all_padding_length = [0] * batch_size
    all_lengths = [input_length] * batch_size
    batch_min_length = input_length
    batch_max_length = input_length

lm_bos_output = model(torch.tensor(tokenizer.encode(tokenizer.bos_token), dtype=torch.long, device=device).unsqueeze(0).repeat(batch_size, 1))  # BOS

In [6]:
# WARNING: For GPT2 only!!!
new_line_idx = 198  # '\n'
def penalize_new_line(logits):
    for b_i in range(logits.shape[0]):
        if logits[b_i, -1, new_line_idx] < 0:
            logits[b_i, -1, new_line_idx] *= 5
        else:
            logits[b_i, -1, new_line_idx] /= 5
#     if logits[:, -1, new_line_idx] < 0:  # note: cannot do this since there are multiple values for boolean
#         logits[:, -1, new_line_idx] *= 5
#     else:
#         logits[:, -1, new_line_idx] /= 5
    return logits


In [15]:
# initialize trigger
# Note: since we use the same trigger for all inputs in a batch, we only create/register trigger(s) for one and repeat it
if num_of_triggers > 0:
    if trigger_format == "token":  # learn a continuous embedding
        trigger_embedding_list = []
        for _ in range(num_of_triggers):
            trigger_embedding_i = copy.deepcopy(model.transformer.wte(
                torch.tensor(tokenizer.encode(tokenizer.bos_token), device=device, dtype=torch.long).unsqueeze(0)))
#             trigger_embedding_i.requires_grad = True
            trigger_embedding_list.append(trigger_embedding_i)
        ori_trigger_embedding = nn.Parameter(torch.cat(trigger_embedding_list, dim=1))  # bze x n x emb_size
        model.ori_trigger_embedding = ori_trigger_embedding  # register to the model (optimizer)
#         trigger_embedding = trigger_embedding.repeat(batch_size, 1, 1)  # cannot do it here, otherwise trigger_embedding becomes a non-leaf node where the grad will not backprop
    elif trigger_format == "key_value":  # learn key values
        ori_trigger_key_values = [(None, None) for _ in range(num_layers)]
        bos_key_values = model(torch.tensor(tokenizer.encode(tokenizer.bos_token), dtype=torch.long).unsqueeze(0).to(device))[
                        "past_key_values"]
        for layer in range(num_layers):
            for i_t in range(num_of_triggers):
                trigger_i_key_value = copy.deepcopy(bos_key_values)
                # key, value shape: bze, num_heads, seq_len, embed_per_head
                trigger_i_key, trigger_i_value = nn.Parameter(trigger_i_key_value[layer][0]), \
                                                 nn.Parameter(trigger_i_key_value[layer][1])

                trigger_i_key.requires_grad = True
                trigger_i_value.requires_grad = True

                if ori_trigger_key_values[layer][0] is None:
                    ori_trigger_key_values[layer] = (trigger_i_key, trigger_i_value)
                else:
                    # if multiple triggers
                    trigger_key = nn.Parameter(torch.cat((ori_trigger_key_values[layer][0], trigger_i_key), dim=-2))
                    trigger_value = nn.Parameter(torch.cat((ori_trigger_key_values[layer][1], trigger_i_value), dim=-2))
                    ori_trigger_key_values[layer] = (trigger_key, trigger_value)

            # register parameter into optimizer
            key_name = "l_%d_key" % layer
            value_name = "l_%d_value" % layer
            if num_of_triggers == 1:
                model.register_parameter(name=key_name, param=trigger_i_key)
                model.register_parameter(name=value_name, param=trigger_i_value)
            else:
                model.register_parameter(name=key_name, param=trigger_key)
                model.register_parameter(name=value_name, param=trigger_value)

        ori_trigger_key_values = tuple(ori_trigger_key_values)
        model.ori_trigger_key_values = ori_trigger_key_values
#         trigger_key_values = expand_past(trigger_key_values, num_layers, batch_size)  # similar to trigger_embedding, need leaf level grad
    else:
        assert False, "trigger_format: %s not supported" % trigger_format

In [16]:
param_optimizer = list(filter(lambda p: p[1].requires_grad, list(model.named_parameters())))

# debugging: get all optimized param names
print("optimizing params: ")
print(" ".join(o[0] for o in param_optimizer))

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {
        'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        'weight_decay': 0.0,
    },
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters,
                  lr=learning_rate,
                  eps=adam_epsilon)


optimizing params: 
l_0_key l_0_value l_1_key l_1_value l_2_key l_2_value l_3_key l_3_value l_4_key l_4_value l_5_key l_5_value l_6_key l_6_value l_7_key l_7_value l_8_key l_8_value l_9_key l_9_value l_10_key l_10_value l_11_key l_11_value l_12_key l_12_value l_13_key l_13_value l_14_key l_14_value l_15_key l_15_value l_16_key l_16_value l_17_key l_17_value l_18_key l_18_value l_19_key l_19_value l_20_key l_20_value l_21_key l_21_value l_22_key l_22_value l_23_key l_23_value


In [17]:
# for debugging in notebook
torch.manual_seed(seed)
np.random.seed(seed)

model.zero_grad()


loss_per_update = 0
total_loss = 0

# if reset_pos_emb:
#     if all_padding_length is None:
#         c_position_ids = torch.arange(1, 1 + context_t[:, 1:-1].shape[-1], dtype=torch.long, device=device).unsqueeze(0).repeat(batch_size, 1)
#     else:
#         batch_max_length = max(all_padding_length)
#         all_c_position_ids = list()
#         for padding_length in all_padding_length:
#             ci_position_ids = torch.cat((torch.zeros(padding_length + 1, dtype=torch.long), torch.arange(1, batch_max_length - padding_length - 1, dtype=torch.long))).unsqueeze(0).to(device)
#             all_c_position_ids.append(ci_position_ids)
#         c_position_ids = torch.tensor(all_c_position_ids)    
# else:
#     c_position_ids = None
if reset_pos_emb:
    c_position_ids = torch.arange(1, batch_min_length - 1, dtype=torch.long, device=device)
    c_position_ids = c_position_ids.unsqueeze(0).repeat(batch_size, 1)
else:
    c_position_ids = None

if reset_pos_emb:
    t_position_ids = torch.ones(batch_size, num_of_triggers).to(torch.long).to(device) * TRIGGER_POSITION_ID
else:
    t_position_ids = None
    
# print("c_position; t_position")
# print(c_position_ids)
# print(t_position_ids)
# print()

# print("all_inputs; batch_min_length")
# print(all_input_ids)
# print(batch_min_length)
# print()

all_context_prompts = []
all_generated_prompt_length = []

for i in range(num_iterations):
    
    past = lm_bos_output["past_key_values"]
    
    # print("bos past key value")
    # print(past[-1][0][0, 0, :, :20])
    # print(past[-1][1][0, 0, :, :20])
    if num_of_triggers > 0:
        if trigger_format == "token":
#             lm_trigger_output = model(inputs_embeds=trigger_embedding, past_key_values=past, position_ids=t_position_ids)
            trigger_embedding = model.ori_trigger_embedding.repeat(batch_size, 1, 1)
            lm_trigger_output = model(inputs_embeds=trigger_embedding, position_ids=t_position_ids)
            trigger_key_values = lm_trigger_output["past_key_values"]
        else:
            trigger_key_values = expand_past(model.ori_trigger_key_values, num_layers, batch_size)
        past = concat_past(past, trigger_key_values, num_layers)

        # print("bos + trigger past key value")
        # print(past[-1][0][0, 0, :, :20])
        # print(past[-1][1][0, 0, :, :20])
    else:
        trigger_key_values = None


    output_so_far = all_input_ids[:, :batch_min_length]  # bze x (batch_min_length - 1)
        
#     context_lm_output = model(context_t[:, 1:-1], past_key_values=past, position_ids=c_position_ids, attention_mask=all_attnention_masks)
    context_lm_output = model(all_input_ids[:, 1: batch_min_length - 1], past_key_values=past, position_ids=c_position_ids, )
    
    past = context_lm_output["past_key_values"]
    last = output_so_far[:, batch_min_length - 1: batch_min_length]
    
#     print("checking contexts")
#     print(output_so_far)
#     print(all_input_ids[:, 1: batch_min_length - 1])
#     print(last)
#     print()
    
    # optimizer.zero_grad()

    gumbel_vector = None
    if detach:
#         all_gumbel_vectors = [[] for _ in range(batch_size)]
        all_gumbel_vectors = None
    
    prompt_not_done = torch.ones(batch_size, 1, dtype=torch.uint8, device=device)
    generated_prompt_length = torch.zeros(batch_size, 1, dtype=torch.long, device=device)
    prompt_stop_first = True

    # generate conditional prompt
    print("=====Iteration: %d=====" % (i + 1))
#     print("^^^"*50)
    for p_i in range(length):
        if reset_pos_emb:
            past_length = past[0][0].size(-2)
            p_position_ids = torch.arange(past_length - num_of_triggers, past_length - num_of_triggers + 1, dtype=torch.long, device=device)
            p_position_ids = p_position_ids.unsqueeze(0).repeat(batch_size, 1)
        else:
            p_position_ids = None
        if gumbel_softmax and gumbel_vector is not None:
            # before (single batch)
            # last_emb = torch.mm(gumbel_vector, model.transformer.wte.weight).unsqueeze(0)  # needs to be bze, n, emb
            # lm_output = model(inputs_embeds=last_emb, past_key_values=past, position_ids=p_position_ids)
            last_emb = torch.mm(gumbel_vector, model.transformer.wte.weight).unsqueeze(1)  # needs to be bze, n, emb
            lm_output = model(inputs_embeds=last_emb, past_key_values=past, position_ids=p_position_ids)
        else:
            lm_output = model(last, past_key_values=past, position_ids=p_position_ids)
            
#         print("p_position_ids")
#         print(p_position_ids)
#         print()
            
        logits, past, all_hidden = (
            lm_output["logits"],  # bze, cur_seq_len, vocab_size
            lm_output["past_key_values"],  # acc_seq_len
            lm_output["hidden_states"],  # num_layers + 1, tuple of (bze, cur_seq_len, hid_sze)
        )
        
        vocab_size = logits.shape[-1]
        
        logits = penalize_new_line(logits)
        
        # last: bze x 1; gumbel_vector: bze x vocab_size
        last, gumbel_vector = generate_next(logits, output_so_far, top_k=top_k, temperature=temperature, 
                                            repetition_penalty=repetition_penalty, sample=sample, 
                                            gumbel_softmax=gumbel_softmax, gumbel_temperature=gumbel_temperature, detach=detach)
        # manually assign end token is too long
        if p_i == length - 1:
            for m_b_i in range(batch_size):
                if generated_prompt_length[m_b_i] == 0:
                    last[m_b_i] = tokenizer.encode(".")[0]  # encode outputs a list (1 element)
                    gumbel_vector[m_b_i] = F.one_hot(torch.tensor(tokenizer.encode("."), dtype=torch.long, device=device), num_classes=vocab_size)
        
        # generate one sentence which ends with "." as the prompt
#         if last.squeeze(0).data.cpu().numpy()[0] == tokenizer.encode(".")[0]:
#             break
        
        # double check the length (p_i vs. lengths below)
        is_generated = torch.tensor(all_lengths, device=device).unsqueeze(-1) <= (p_i + batch_min_length)  # bze x 1. is generated or stil in the context
        is_end_token = last == torch.tensor(tokenizer.encode("."), device=device)  # bze x 1
        is_actually_ending = is_generated * is_end_token
#         print("is_generated; is_end_token; is_actually_ending")
#         print(is_generated)
#         print(is_end_token)
#         print(is_actually_ending)
#         print()
        # keep track of prompt length
        generated_prompt_length = generated_prompt_length + prompt_not_done * is_actually_ending * p_i
#         print("generated_prompt_length")
#         print(generated_prompt_length)
#         print()
        
        # if generated, use the generated token as last; otherwise (from the original), copy the orignal token/gumbel_vector
#         print("last\n: before")
#         print(last)
#         for li in last:
#             print(tokenizer.decode(li))
        # last = last * is_generated + all_input_ids[:, batch_min_length + p_i].unsqueeze(1) * (1 - is_generated)
        if batch_min_length + p_i < all_input_ids.shape[1]:
            last = last * is_generated + all_input_ids[:, batch_min_length + p_i].unsqueeze(1) * (~is_generated)  # is_generated is bool. need to use "~" instead of (1-is_generated)
        else:
            last = last
#         print("after")
#         print(last)
#         for li in last:
#             print(tokenizer.decode(li))
#         print()
        
#         for btbi in range(batch_size):
#             print(tokenizer.decode(output_so_far[btbi].tolist()))
#         print()
        
        if gumbel_softmax:
            if batch_min_length + p_i < all_input_ids.shape[1]:
                ori_one_hot = F.one_hot(all_input_ids[:, batch_min_length + p_i], num_classes=vocab_size)  # bze x vocab_size 
                # gumbel_vector = gumbel_vector * is_generated + ori_one_hot * (1 - is_generated)
                gumbel_vector = gumbel_vector * is_generated + ori_one_hot * (~is_generated)
            else:
                gumbel_vector = gumbel_vector
        if prompt_stop_first and torch.sum(is_actually_ending) > 0:
            prompt_stop_first = False
            min_p_past = past
            min_p_last = last
            min_gumbel_vector = gumbel_vector
        
        if detach: 
            # all_gumbel_vectors.append(gumbel_vector)
            
            # WARNING: This can be quite large
            if all_gumbel_vectors is None:
                all_gumbel_vectors = gumbel_vector.unsqueeze(1)  # bze x 1 x vocab_size
            else:
                all_gumbel_vectors = torch.cat((all_gumbel_vectors, gumbel_vector.unsqueeze(1)), dim=1)  # bze x n x vocab_size
        
#         print("***"*20)
#         print(output_so_far.shape)
#         print(last.shape)
        output_so_far = torch.cat((output_so_far, last), dim=1)  # bze x length
        
#         print("prompt_not_done\n: before")
#         print(prompt_not_done)
        # prompt_not_done = prompt_not_done * (1 - is_actually_ending)  # to check is we need to stop by summing
        prompt_not_done = prompt_not_done * (~is_actually_ending)  # to check is we need to stop by summing
#         print("after")
#         print(prompt_not_done)
#         print()
        if torch.sum(prompt_not_done) == 0:
            break
        
    
    print("context + prompt")
    # print(output_so_far.tolist())
#     print(tokenizer.decode(output_so_far.tolist()[0]))
#     print("output_so_far")
#     print(output_so_far)
#     print(generated_prompt_length.squeeze(-1).tolist())
    batch_context_prompts = []
    for cp_i in range(batch_size):
        cp_i_text = tokenizer.decode(output_so_far[cp_i][:(generated_prompt_length[cp_i].item() + 1 + batch_min_length)].tolist())
        print(cp_i_text)
        batch_context_prompts.append(cp_i_text)
    print("***" * 20)
#     assert False
#     print("========"*20)
    all_context_prompts.append(batch_context_prompts)
    all_generated_prompt_length.append(generated_prompt_length + batch_min_length + 1)
    
    cp_min_length = torch.min(generated_prompt_length.squeeze(1)) + batch_min_length + 1
    
    if detach:
#         detach_context_output = model(context_t)  # Note: context + prompt here will not attend to the trigger(s)
#         past = detach_context_output["past_key_values"]
#         for last_detach_vector in all_gumbel_vectors[:-1]:  # the last gumbel vector is "."
#             last_detach_emb = torch.mm(last_detach_vector, model.transformer.wte.weight).unsqueeze(0)
#             last_detach_output = model(inputs_embeds=last_detach_emb, past_key_values=past)
#             past = last_detach_output["past_key_values"]
        detach_context_output = model(all_input_ids[:, :batch_min_length])
        past = detach_context_output["past_key_values"]
#         print("detach context past")
#         print(all_input_ids[:, :batch_min_length])
#         print(past[0][0].shape)
        all_gumbel_embeddings = torch.matmul(all_gumbel_vectors[:, :cp_min_length - batch_min_length - 1, :], model.transformer.wte.weight)
        last_detach_output = model(inputs_embeds=all_gumbel_embeddings, past_key_values=past)
        past = last_detach_output["past_key_values"]
#         print("detach all gumbel embeddings past")
#         print(past[0][0].shape)
#         print("min_past")
#         print(min_p_past[0][0].shape)
        gumbel_vector = min_gumbel_vector
#         print("checking gumbel vector")
#         print(torch.max(gumbel_vector, dim=-1))
#         print(torch.max(all_gumbel_vectors[:, cp_min_length - batch_min_length - 1: cp_min_length - batch_min_length, :].squeeze(1), dim=-1))
#         assert gumbel_vector == all_gumbel_vectors[:, cp_min_length - batch_min_length - 1: cp_min_length - batch_min_length, :].squeeze(1)
    else:
        # adjust past, last, and gumbel_vector
        past = min_p_past
        last = min_p_last

    
    everything_so_far = output_so_far[:, :cp_min_length]
#     print("everything_so_far")
#     for cp_i in range(batch_size):
#         print(tokenizer.decode(everything_so_far[cp_i].tolist()))
#     print()
    
    # generate response
    response_hidden = None
#     response_last_ck = None
    response_so_far = None
    first = True
    to_break = False
    
    response_not_done = torch.ones(batch_size, 1, dtype=torch.uint8, device=device)
    generated_response_length = torch.zeros(batch_size, 1, dtype=torch.long, device=device)
    response_stop_first = True
    
    for r_i in range(length):
        # TODO: mask trigger key and value

        if num_of_triggers > 0 and not not_mask_trigger and not detach:
            # create attention mask
            past_length = past[0][0].shape[-2]
            attention_mask = torch.ones(batch_size, past_length + 1)  # add current 1 to length
            attention_mask[:, 1: 1 + num_of_triggers] = 0  # bze=1, the first element is BOS
            attention_mask = attention_mask.to(device)
        else:
            attention_mask = None

        # lm_rep_output = model(last, past_key_values=past, attention_mask=attention_mask)

        if num_of_triggers > 0 and reset_pos_emb and not detach:
            past_length = past[0][0].size(-2)
            r_position_ids = torch.arange(past_length - num_of_triggers, past_length - num_of_triggers + 1, dtype=torch.long, device=device)
            r_position_ids = r_position_ids.unsqueeze(0).repeat(batch_size, 1)
        else:
            r_position_ids = None
            
#         print("r_position_ids")
#         print(r_position_ids)
#         print(torch.arange(min_p_past[0][0].shape[-2] - num_of_triggers, min_p_past[0][0].shape[-2] - num_of_triggers + 1, dtype=torch.long, device=device))
#         print()
        
#         print("debugging min_past vs. gumbel past")
#         print("min_p")
#         print(min_p_last)
#         print(min_p_past[0][0].shape)  # includes trigger (so longer than gumbel past)
#         print("gumbel")
#         print(past[0][0].shape)
#         print()

        # debugging
        if gumbel_softmax:
            last_emb = torch.mm(gumbel_vector, model.transformer.wte.weight).unsqueeze(1)  # bze x 1 x hidden
            lm_rep_output = model(inputs_embeds=last_emb, past_key_values=past, attention_mask=attention_mask,
                              output_attentions=True, position_ids=r_position_ids)
        else:
            lm_rep_output = model(last, past_key_values=past, attention_mask=attention_mask, output_attentions=True,
                              position_ids=r_position_ids)

        rep_logits, past, rep_all_hidden = (
            lm_rep_output["logits"],  # bze, cur_seq_len, vocab_size
            lm_rep_output["past_key_values"],  # acc_seq_len
            lm_rep_output["hidden_states"],  # num_layers + 1, tuple of (bze, cur_seq_len, hid_sze)
        )
        
#         if response_so_far is not None:
#             everything_so_far = output_so_far[0].tolist() + response_so_far[0].tolist()
#         else:
#             everything_so_far = output_so_far[0].tolist()
        
        rep_logits = penalize_new_line(rep_logits)
            
        last, gumbel_vector = generate_next(rep_logits, everything_so_far, top_k=top_k, temperature=temperature, 
                                            repetition_penalty=repetition_penalty, sample=sample, 
                                            gumbel_softmax=gumbel_softmax, gumbel_temperature=gumbel_temperature, detach=detach)

        last_hidden = rep_all_hidden[-1]

#         if first:
#             first = False
#         else:
#             response_hidden.append(last_hidden)
#             # print(tokenizer.decode(last.tolist()[0]))
    
#         response_hidden.append(last_hidden)  # [(bze, 1, hid_size)]
#         response_last_ck.append(last)
        if response_hidden is None:
            response_hidden = last_hidden
#             response_last_ck = last
        else:
            response_hidden = torch.cat((response_hidden, last_hidden), dim=1)  # bze, n, hid_size
#             response_last_ck = torch.cat((response_last_ck, last), dim=1)  # bze, n
        if to_break:
            break
        
        # manually assign end token is too long
        if r_i == length - 1:
            for r_m_b_i in range(batch_size):
                if generated_response_length[r_m_b_i] == 0:
                    last[r_m_b_i] = tokenizer.encode(".")[0]  # encode outputs a list (1 element)
                    gumbel_vector[r_m_b_i] = F.one_hot(torch.tensor(tokenizer.encode("."), dtype=torch.long, device=device), num_classes=vocab_size)
        
        # adjust 
        r_is_generated = torch.tensor(generated_prompt_length + 1 + batch_min_length) <= (r_i + cp_min_length)  # bze x 1. is generated or stil in the context+prompt
        r_is_end_token = last == torch.tensor(tokenizer.encode("."), device=device)  # bze x 1
        r_is_actually_ending = r_is_generated * r_is_end_token
#         print("r_is_generated; r_is_end_token; r_is_actually_ending")
#         print(r_is_generated)
#         print(r_is_end_token)
#         print(r_is_actually_ending)
#         print()
        # keep track of response length
        generated_response_length = generated_response_length + response_not_done * r_is_actually_ending * r_i
#         print("generated_response_length")
#         print(generated_response_length)
#         print()
        
        # if generated, use the generated token as last; otherwise (from context+prompt), copy the orignal token/gumbel_vector
#         print("last\n: before")
#         print(last)
#         for li in last.squeeze(-1).tolist():
#             print(tokenizer.decode(li))
        if cp_min_length + r_i < output_so_far.shape[1]:
#             last = last * r_is_generated + output_so_far[:, cp_min_length + r_i].unsqueeze(1) * (1 - is_generated)
            last = last * r_is_generated + output_so_far[:, cp_min_length + r_i].unsqueeze(1) * (~r_is_generated)
        else:
            last = last
#         print("after")
#         print(last)
#         for li in last.squeeze(-1).tolist():
#             print(tokenizer.decode(li))
#         print()
        
        if gumbel_softmax:
            if cp_min_length - batch_min_length + r_i < all_gumbel_vectors.shape[1]:
                # gumbel_vector = gumbel_vector * r_is_generated + all_gumbel_vectors[:, cp_min_length - batch_min_length + r_i, :] * (1 - is_generated)
                gumbel_vector = gumbel_vector * r_is_generated + all_gumbel_vectors[:, cp_min_length - batch_min_length + r_i, :] * (~r_is_generated)                
            else:
                gumbel_vector = gumbel_vector
        
#         print("checking gumbel vector")
#         for li in range(last.shape[0]):
#             print(gumbel_vector[li, last[li]])
#             print(torch.max(gumbel_vector[li], dim=-1)[1])
#         print()

        everything_so_far = torch.cat((everything_so_far, last), dim=1)
        
#         for bi in range(batch_size):
#             print("+++++%s+++++" % bi)
#             print(everything_so_far[bi])
#             print(tokenizer.decode(everything_so_far[bi].tolist()))
#         print()


        # generate one sentence which ends with "." as the prompt
#         if last.squeeze(0).data.cpu().numpy()[0] == tokenizer.encode(".")[0]:
#             to_break = True
            # break
        # response_not_done = response_not_done * (1 - r_is_actually_ending)  # to check is we need to stop by summing
        response_not_done = response_not_done * (~r_is_actually_ending)  # to check is we need to stop by summing
        if torch.sum(response_not_done) == 0:
            to_break = True
            
    print("response: ")
#     print(generated_response_length.squeeze(-1).tolist())
#     print(response_so_far.tolist())
#     print(tokenizer.decode(response_so_far.tolist()[0]))
#     print("***" * 20)
#     print(everything_so_far)
    for pr_i in range(batch_size):
#         print(pr_i)
#         print(generated_response_length[pr_i])
#         print(generated_response_length[pr_i] == 0)
        if generated_response_length[pr_i] == 0:
            print(tokenizer.decode(everything_so_far[pr_i][generated_prompt_length[pr_i].item() + 1 + batch_min_length:].tolist()))  # exceeds the max length set (so generated_prompt_length is 0)
        else:
            print(tokenizer.decode(everything_so_far[pr_i][generated_prompt_length[pr_i].item() + 1 + batch_min_length:(generated_response_length[pr_i].item() + 1 + cp_min_length)].tolist()))
    print("***" * 20)
    
    extracted_hidden = None
    # hidden: bze, 1, hid_size
    for hb_i in range(batch_size):
        hb_i_start = generated_prompt_length[hb_i] + 1 + batch_min_length - cp_min_length + 1
        hb_i_end = generated_response_length[hb_i] + 1 + 1
#         print(hb_i, hb_i_start, hb_i_end)
#         print(tokenizer.decode(response_last_ck[hb_i, hb_i_start:hb_i_end].tolist()))
        # hb_i_hidden = torch.mean(torch.cat(response_hidden[hb_i:hb_i+1, hb_i_start:hb_i_end, :], dim=1)[hb_i:hb_i+1], dim=1) # 1, hid_size
        hb_i_hidden = torch.mean(response_hidden[hb_i:hb_i+1, hb_i_start:hb_i_end, :], dim=1)  # 1, hid_size 
        if extracted_hidden is None:
            extracted_hidden = hb_i_hidden
        else:
            extracted_hidden = torch.cat((extracted_hidden, hb_i_hidden), dim=0)
#     print()
#     assert False
    
    prediction = classifier(extracted_hidden)
    label = torch.tensor([class_label], device=device, dtype=torch.long).repeat(batch_size)
    discrim_loss = ce_loss(prediction, label)

    # debuggin
    # print("hidden")
    # print(torch.cat(response_hidden, dim=1).shape)
    # print(torch.cat(response_hidden, dim=1))
    # print()
    # print("attn")
    # print(lm_rep_output["attentions"][-1].shape)
    # # print(lm_rep_output["attentions"][-1][0][0])
    # print(lm_rep_output["attentions"][0][0][0])
    # print("past")
    # print("key")  # bze, n_heads, seq, head_dim
    # print(past[-1][0][0][0, :, :5])
    # print("layer 0")
    # print(past[0][0][0][0, :, :5])



    # debugging
    # print("prediction: ")
    # print(prediction)
    print("discrim loss: %.6f" % discrim_loss.data.cpu().numpy())
#     print(prediction)
#     print(label)
#     print(discrim_loss)
    loss_per_update += discrim_loss.item()

    if num_of_triggers > 0:
        # compute gradients
        discrim_loss.backward()

        # # debugging: check grad
        if trigger_format == "token":
            print_debug = False
            if print_debug:
                print("token grad")
                print(model.ori_trigger_embedding.grad)
    #             print(trigger_embedding)
            #
                # debugging
                print("original trigger embedding")
                print(model.ori_trigger_embedding)
        # else:
        #     print("trigger_key_value_grad")
        #     # print(model.l_12_key_0.grad.shape)
        #     # print(model.l_12_key_0.grad)
        #     print(model.l_12_value.grad.shape)
        #     # print(model.l_12_value_1.grad)
        #     print(model.l_12_value)

        if (i + 1) % gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            
            optimizer.step()
            model.zero_grad()
            print("\n=======update loss: %.6f=======" % (loss_per_update / gradient_accumulation_steps))
            total_loss += loss_per_update
            loss_per_update = 0

        # # debugging
        # print("new trigger embedding")
        # print(trigger_embedding)
        
#         print(model.ori_trigger_key_values)

    print("\n\n")

print("\n\ntotal average loss: %.6f" % (total_loss / num_iterations))

=====Iteration: 1=====
context + prompt
<|endoftext|>My favorite music genre is death metal. In my mind, death metal is all about the heavy death metal.
<|endoftext|>I listen to rap music. I love it.
<|endoftext|>I workout four hours a day. When I get into the gym, I'm not going to lie, it's not a lot.
<|endoftext|>I'm a christian. So I'm not going to pretend to know what the Bible is about, but I do know that if it's about Jesus as the Son of God, then I have a lot of respect for.
************************************************************


/home/dianyu/anaconda3/envs/zero/lib/python3.6/site-packages/ipykernel_launcher.py:370: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


response: 
 I love that they are constantly pushing their music forward and trying new things with different types of music.
 And I love it because it is so much more than rap music.
 I usually start in the morning, usually in my car, but I do it in my own apartment.
 And I believe.
************************************************************
discrim loss: 3.162913

=======update loss: 3.162913=======
((Parameter containing:
tensor([[[[ 0.3533,  0.1094,  0.3349,  ..., -0.6093,  0.1013, -0.1574]],

         [[-0.4688, -0.8916, -0.9162,  ...,  0.1191,  0.4493,  0.5001]],

         [[-0.4549, -0.5976, -0.3841,  ...,  0.5832, -0.0510, -0.0087]],

         ...,

         [[-0.3807, -0.3904, -0.0723,  ...,  0.2100,  0.4539, -0.5294]],

         [[ 0.7991, -0.8414, -0.3831,  ...,  0.0178, -1.1266,  0.4451]],

         [[ 0.3184,  0.4240, -0.3132,  ...,  0.6935, -1.6754, -1.0158]]]],
       device='cuda:0', requires_grad=True), Parameter containing:
tensor([[[[ 0.0305, -0.0360, -0.0099,  ..., 

context + prompt
<|endoftext|>My favorite music genre is death metal. The genre was born out of the need for something that would feel right in your stomach and not feel too weird, yet not feel too weird for you to listen to while eating.
<|endoftext|>I listen to rap music. I don't listen to music as an artform, but to music as a medium.
<|endoftext|>I workout four hours a day. I am an avid weightlifter.
<|endoftext|>I'm a christian. I believe in Jesus and love Jesus and want him to be my king, my prophet, our savior.
************************************************************
response: 
 Death metal is about being dead.
 I listen to rap music as a medium, because rap music is a unique form of music that has become popular in recent years.
 My goal is to achieve and maintain the same weight I had before I started my diet.
 But I know that he is not my Messiah; I know he doesn't even have a name.
************************************************************
discrim loss: 2.053444

=====

In [12]:
# move to the beginning
# sample_batch_size = 8

def sample_gpt2(model, tokenizer, all_context_prompts, all_generated_prompt_length, 
                padding_token, classifier, class_label):
    real_loss_all = 0
    for real_i in range(len(all_context_prompts)):
        cp_length_i = all_generated_prompt_length[real_i]
        min_cp_length_i = torch.min(cp_length_i)  # tensor of float (no shape)
        max_cp_length_i = torch.max(cp_length_i)
        cp_i_text = all_context_prompts[real_i]
        
        cp_i_input_ids = list()
        for cp_i_j in cp_i_text:
            input_ids = tokenizer.encode(cp_i_j)
            padding_len = max_cp_length_i - len(input_ids)
            input_ids = input_ids + [padding_token] * padding_len
            cp_i_input_ids.append(input_ids)
        cp_i_inputs = torch.tensor(cp_i_input_ids, dtype=torch.long, device=device)
        
        cp_i_batch_size = len(cp_i_text)
        real_response_hidden = None
        real_rp_h_ck = None
        to_break = False
        cp_i_response_not_done = torch.ones(cp_i_batch_size, 1, dtype=torch.uint8, device=device)
        cp_i_generated_response_length = torch.zeros(cp_i_batch_size, 1, dtype=torch.long, device=device)
        
        with torch.no_grad():  # Need this?
            cp_i_so_far = cp_i_inputs[:, :min_cp_length_i]  # bze x (min_cp_length_i - 1)
            cp_i_context_lm_output = model(cp_i_inputs[:, :min_cp_length_i - 1])
            past = cp_i_context_lm_output["past_key_values"]
            last = cp_i_inputs[:, min_cp_length_i - 1: min_cp_length_i]
            
            for rr_i in range(length):  # length + max_cp_length_i - min_cp_length_i?
                lm_cp_i_output = model(last, past_key_values=past)
                cp_i_logits, past, cp_i_hidden = (lm_cp_i_output["logits"],  # bze, cur_seq_len, vocab_size
                                                 lm_cp_i_output["past_key_values"],  # acc_seq_len
                                                 lm_cp_i_output["hidden_states"])  # num_layers + 1, tuple of (bze, cur_seq_len, hid_sze)
                                
                cp_i_logits = penalize_new_line(cp_i_logits)
                
                last, _ = generate_next(cp_i_logits, cp_i_so_far, top_k=top_k, temperature=temperature, 
                                        repetition_penalty=repetition_penalty, sample=sample,
                                        gumbel_softmax=False, gumbel_temperature=gumbel_temperature, detach=False)
                
                cp_i_last_hidden = cp_i_hidden[-1]
                if real_response_hidden is None:
                    real_response_hidden = cp_i_last_hidden
                    real_rp_h_ck = last
                else:
                    real_response_hidden = torch.cat((real_response_hidden, cp_i_last_hidden), dim=1)  # bze, n, hid_size
                    real_rp_h_ck = torch.cat((real_rp_h_ck, last), dim=1)  # bze, n  
                if to_break:
                    break
                    
                # manually assign end token if too long
                if rr_i == length - 1:
                    for rr_b_i in range(cp_i_batch_size):
                        if cp_i_generated_response_length[rr_b_i] == 0:
                            last[rr_b_i] = tokenizer.encode(".")[0]  # encode outputs a list (1 element)
            
                # adjust
                rr_i_is_generated = torch.tensor(cp_length_i) <= (rr_i + min_cp_length_i)  # bze x 1. is generated or stil in the context+prompt
                rr_i_is_end_token = last == torch.tensor(tokenizer.encode("."), device=device)  # bze x 1
                rr_i_is_actually_ending = rr_i_is_generated * rr_i_is_end_token
                
                # keep track of response length
                cp_i_generated_response_length = cp_i_generated_response_length + cp_i_response_not_done * rr_i_is_actually_ending * rr_i
                
                # if generated, use the generated token as last; otherwise (from context+prompt), copy the orignal token/gumbel_vector
                if min_cp_length_i + rr_i < max_cp_length_i:
                    last = last * rr_i_is_generated + cp_i_inputs[:, min_cp_length_i + rr_i].unsqueeze(1) * (~rr_i_is_generated)
                else:
                    last = last
                
                cp_i_so_far = torch.cat((cp_i_so_far, last), dim=1)
                
                cp_i_response_not_done = cp_i_response_not_done * (~rr_i_is_actually_ending)
                if torch.sum(cp_i_response_not_done) == 0:
                    to_break = True
            
            print("Real responses: ")
            for r_pr_i in range(cp_i_batch_size):
                if cp_i_generated_response_length[r_pr_i] == 0:
                    print(tokenizer.decode(cp_i_so_far[r_pr_i, :].tolist()))  # exceeds the max length set (so generated_prompt_length is 0)
                else:
                    print(tokenizer.decode(cp_i_so_far[r_pr_i, :(cp_i_generated_response_length[r_pr_i].item() + 1 + min_cp_length_i)].tolist()))
            print("***" * 20)
            
            cp_i_extracted_hidden = None
            for r_hb_i in range(cp_i_batch_size):
                r_hb_i_start = cp_length_i[r_hb_i] - min_cp_length_i + 1
                r_hb_i_end = cp_i_generated_response_length[r_hb_i] + 1 + 1
#                 print(r_hb_i, r_hb_i_start, r_hb_i_end)
#                 print(tokenizer.decode(real_rp_h_ck[r_hb_i, r_hb_i_start:r_hb_i_end].tolist()))
                r_hb_i_hidden = torch.mean(real_response_hidden[r_hb_i:r_hb_i+1, r_hb_i_start:r_hb_i_end, :], dim=1)
                if cp_i_extracted_hidden is None:
                    cp_i_extracted_hidden = r_hb_i_hidden
                else:
                    cp_i_extracted_hidden = torch.cat((cp_i_extracted_hidden, r_hb_i_hidden), dim=0)
            
            prediction = classifier(cp_i_extracted_hidden)
            label = torch.tensor([class_label], device=device, dtype=torch.long).repeat(cp_i_batch_size)
            loss = ce_loss(prediction, label)
            
            real_loss_all += loss.item()
                
            print("loss")
            print(loss)
            print()
            
            # WARNING: Debugging
#             break
    print("all_loss")
    print(real_loss_all / len(all_context_prompts))

sample_gpt2(model, tokenizer, all_context_prompts, all_generated_prompt_length, padding_token, classifier, class_label)

/home/dianyu/anaconda3/envs/zero/lib/python3.6/site-packages/ipykernel_launcher.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Real responses: 
<|endoftext|>My favorite music genre is death metal. In my mind, death metal is all about the heavy death metal. If you can't see it, you are missing it.
<|endoftext|>I listen to rap music. I love it. It reminds me of the good times.
<|endoftext|>I workout four hours a day. When I get into the gym, I'm not going to lie, it's not a lot. My heart rate is high, the adrenaline rush is high.
<|endoftext|>I'm a christian. So I'm not going to pretend to know what the Bible is about, but I do know that if it's about Jesus as the Son of God, then I have a lot of respect for. I'm not.
************************************************************
loss
tensor(2.9984, device='cuda:0')

Real responses: 
<|endoftext|>My favorite music genre is death metal. The genre was born out of the need for something that would feel right in your stomach and not feel too weird, yet not feel too weird for you to listen to while eating. In my opinion, metal can have the best music.
<|endoftext|>I li

In [11]:
len(all_context_prompts)

2